# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Import tools and libraries
from dotenv import load_dotenv
from tavily import TavilyClient
import os
import json
import re
from pydantic import BaseModel
from openai import OpenAI
import chromadb
from lib.tooling import tool
# from langchain.tools import tool

from lib.agents import Agent

In [2]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:

# ------------------------------
# Chroma setup
# ------------------------------
chroma_client = chromadb.PersistentClient(path="chromadb")

# ------------------------------
# RETRIEVE_GAME TOOL
# ------------------------------
@tool
def retrieve_game(query: str, n_results: int = 3):
    """
    Semantic search: Finds the most relevant games from the vector DB.

    Args:
        query (str): A question about the game industry.

    Returns:
        List[Dict]: Each result contains:
            - Platform (e.g., Game Boy, PS5, Xbox 360)
            - Name (name of the game)
            - YearOfRelease
            - Description
    """
    print("🔎 Retrieving from vector DB...")
    try:
        collection = chroma_client.get_collection("games_collection_new")

        results = collection.query(
            query_texts=query,
            n_results=n_results,
            include=["documents", "metadatas"]
        )

        metadatas = results.get("metadatas", [[]])[0]
        documents = results.get("documents", [[]])[0]

        output = []
        for meta, doc in zip(metadatas, documents):
            output.append({
                "Platform": meta.get("Platform"),
                "Name": meta.get("Name"),
                "YearOfRelease": meta.get("YearOfRelease"),
                "Description": meta.get("Description"),
                "document": doc
            })

        return {
            "data": output
        }

    except Exception as e:
        return [{"error": str(e)}]


#### Evaluate Retrieval Tool

In [4]:

class EvaluationReport(BaseModel):
    useful: bool
    description: str

@tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    """
    LLM judge to evaluate whether retrieved docs are sufficient.
    """
    print("🧠 Evaluating retrieval...")

    prompt = f"""
You are an evaluation assistant. Your task is to decide whether the retrieved documents are sufficient to answer the user's question.

Question:
{question}

Retrieved Documents:
{json.dumps(retrieved_docs, indent=2)}

Return ONLY valid JSON in this format:

{{"useful": true, "description": "..."}}
"""

    client = OpenAI(api_key=OPENAI_API_KEY, base_url=os.getenv("OPENAI_BASE_URL"))

    resp = client.chat.completions.create(
        model=os.getenv("EVAL_MODEL", "gpt-4o-mini"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )

    raw = resp.choices[0].message.content

    # --- JSON extraction ---
    try:
        parsed = json.loads(raw)
    except:
        match = re.search(r"\{[\s\S]*\}", raw)
        if not match:
            return {
                "useful": False,
                "description": f"Failed to extract JSON. Raw: {raw}"
            }
        parsed = json.loads(match.group(0))

    try:
        report = EvaluationReport.model_validate(parsed)
    except Exception as e:
        return {
            "useful": False,
            "description": f"Invalid JSON schema: {str(e)}. Raw: {parsed}"
        }

    return report.model_dump()



#### Game Web Search Tool

In [5]:

@tool
def game_web_search(question: str, max_results: int = 3) -> dict:
    """
    Uses Tavily client to search the web for a gaming-related question.
    Returns a formatted, human-readable summary of sources.
    """
    print("🌐 Searching the web...")
    tavily = TavilyClient(api_key=TAVILY_API_KEY)
    resp = tavily.search(
        query=question,
        include_answer=True,
        max_results=max_results
    )

    return {
        "answers": resp.get("results", []),
    }


### Agent

In [6]:
tools = [
    retrieve_game,
    evaluate_retrieval,
    game_web_search
]

uda_agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions=(
        """
        You are a GameAgent, when a game-related question is asked, you MUST follow this pipeline:
        1. Retrieve relevant documents from the local game database using the retrieve_game tool.
        2. Evaluate the quality of the retrieved documents using the evaluate_retrieval tool.
        3. If the retrieved documents are sufficient, provide a concise answer based on them.
        4. If the documents are insufficient, search the web using the game_web_search tool. IF you use the `game_web_search` tool, then your FINAL ANSWER MUST follow this exact format:
         Based on web search results:
         - According to {title} ({url}): "{content}"
         - According to {title} ({url}): "{content}"

         The {title} MUST come from the web search result field "title".
         The {content} MUST come from the result field "content".
         The {url} MUST come from the field "url"
         """
    ))

In [7]:


queries = [
    "What is the genre of the game Super Mario 64?",
    "When was Super Smash Bros. Melee released?",
    "Who developed the game Zula?",
]

for q in queries:
    print("\n========================================")
    print("QUESTION:", q)
    print("========================================\n")

    data = uda_agent.invoke(q)
    result = data.get_final_state()
    last_answer = next(
        (m.content for m in reversed(result["messages"])
         if m.role == "assistant" and m.content),
        None
    )
    print(last_answer)



QUESTION: What is the genre of the game Super Mario 64?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🔎 Retrieving from vector DB...


C:\Projects\building-agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
🧠 Evaluating retrieval...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
The genre of the game **Super Mario 64** is **3D platformer**. It is known for setting new standards in the genre during its release in 1996.

QUESTION: When was Super Smash Bros. Melee released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🔎 Retrieving from vector DB...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
🧠 Evaluating retrieval...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
**Super Smash Bros. Melee** was released in **2001**.

QUESTION: Who developed the game Zula?

[StateMachine] Starting: __entry__
[StateMachine] Exe

### (Optional) Advanced

In [8]:
import uuid
from datetime import datetime
from chromadb.utils import embedding_functions

# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
long_term_memory = chroma_client.get_or_create_collection(name="long_term_memory", embedding_function=embedding_fn)


@tool
def store_memory(text: str):
    """
    Saves any text into Chroma vector memory with timestamp.
    """
    print("🧠 Storing memory...")
    memory_id = str(uuid.uuid4())
    long_term_memory.add(
        documents=[text],
        ids=[memory_id],
        metadatas=[{"timestamp": datetime.now().isoformat()}]
    )
    return f"Memory saved: {text}"

@tool
def recall_relevant_memory(query: str, k: int = 1):
    """
    Retrieves the top-k most relevant memories for the agent.
    Returns a structured object including the source for traceability.
    """
    print("🧠 Recalling relevant memories...")

    results = long_term_memory.query(
        query_texts=[query],
        n_results=k
    )

    # No results case — still return a source tag
    if len(results["documents"]) == 0 or len(results["documents"][0]) == 0:
        return {
            "source": "long_term_memory",
            "found": False,
            "memories": [],
            "text": "No relevant memory found."
        }

    # Extract as list of strings
    memories = results["documents"][0]

    return {
        "source": "long_term_memory",
        "found": True,
        "memories": memories,
        "text": "\n".join([f"- {m}" for m in memories])
    }


In [9]:
uda_agent_with_memory = Agent(
    model_name="gpt-4o-mini",
    tools=tools + [store_memory, recall_relevant_memory],
    instructions=(
        """
        You are a GameAgent with pre-defined tools and long-term memory.
        When a game-related question is asked, you MUST follow this pipeline:
        1. Recall relevant memories using the recall_relevant_memory tool. If you find any relevant memories, use them to help answer the question.
        2. Retrieve relevant documents from the local game database using the retrieve_game tool.
        3. Evaluate the quality of the retrieved documents using the evaluate_retrieval tool.
        4. If the retrieved documents are sufficient, provide a concise answer based on them.
        5. If the documents are insufficient, search the web using the game_web_search tool. IF you use the `game_web_search` tool, then your FINAL ANSWER MUST follow this exact format:
         Based on web search results:
         - According to {title} ({url}): "{content}"
         - According to {title} ({url}): "{content}"

         The {title} MUST come from the web search result field "title".
         The {content} MUST come from the result field "content".
         The {url} MUST come from the field "url"
        6. After providing an answer, store any useful information into long-term memory using the store_memory tool.
         """
    ))


In [10]:
queries = [
    "When was the game Gran Turismo released?",
    "When was the game Zula released?",
    "Where can I play the game Zula?",
    "For what platform was the game Kinect Adventures released?",
    "Who developed the game Minecraft?",
    "Where can I play the game Zula?"

]

for q in queries:
    print("\n========================================")
    print("QUESTION:", q)
    print("========================================\n")

    data = uda_agent_with_memory.invoke(q)
    messages = data.get_final_state()["messages"]
    result = []
    for msg in reversed(messages):
        if msg.role == "assistant" and msg.content:
            result = msg.content
            break
    print(result)






QUESTION: When was the game Gran Turismo released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🧠 Recalling relevant memories...
🔎 Retrieving from vector DB...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
🧠 Evaluating retrieval...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
The game Gran Turismo was released in 1997 for the PlayStation 1. It is a realistic racing simulator known for featuring a wide array of cars and tracks, setting a new standard for the genre. 

I will store this information for future reference.

QUESTION: When was the game Zula released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🧠 Recalling relevant memories...
🔎 Retrieving from vector DB...
[StateMachine] Executing step: